# 2-D Diffusionsgleichung (Stationär)

## Einführung

Unter der Annahme, dass sich die Temperatur nicht mehr mit der Zeit ändert, also das Temperaturfeld stationär ist, vereinfacht sich die Wärmeleitgleichung in eine reine Diffusionsgleichung in der Form einer sogenannte Poissongleichung (nur radiogene Elemente):

$\begin{equation}
0 = \frac{\partial}{\partial{x}} \left(k_{x} \frac{\partial{T}}{\partial{x}}\right) + \frac{\partial}{\partial{y}} \left(k_{y} \frac{\partial{T}}{\partial{y}}\right) + \rho H_r,
\end{equation}$

wobei $\rho$ die Dichte, $k_{x,y}$ die Wärmeleitfähigkeit in x- und y-Richtung, und $H_r$ die radiogene Wärmeproduktion pro Masse [W/kg] ist. Vereinfachen wir nun die Gleichung noch ein wenig und nehmen an, dass die thermischen Parameter (hier, vor allem die Wärmeleitfähigkeit $k$) **isotrop** und **konstant** sind:

$\begin{equation}
0 = \left( \frac{\partial^2{T}}{\partial{x}^2} + \frac{\partial^2{T}}{\partial{y}^2} \right) + \frac{Q}{k},
\end{equation}$

wobei $Q = \rho H_r$ die Wärmeproduktionsrate pro Volumen [W/m^3] ist. 

## Das Problem
Nehmen wir an, dass die Gleichung $(2)$ in einem 2D rechteckigen Gebiet mit der Seitenlänge ($L$) 4000 m und der Tiefe ($H$) 2000 m definiert sei. In dem Gebiet sei ein Körper mit hoher Wärmeproduktionsrate eingelagert, der die Eckkoordinaten (x, z) in km habe: (1900, 900), (2100, 900), (2100, 1100) und (1900, 1100). Die Wärmeproduktionsrate $Q$ betrage 0.3 W/m<sup>3</sup> in dem Körper und 0 außerhalb. Die Wärmeleitfähigkeit ($k$) betrage 6.5 W/(K m). Als Randbedingung (Dirichlet) sei T = 0 °C auf dem gesamten Rand zu setzen.

<img src="../Figures/Exercise04_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau

Die beschriebene Konfiguration beschreibt in etwa die Situation hochradioaktiver Endlagerung in einem Salzstock. Dort sollen radioaktive Behälter mit einem Durchmesser von 1 m in 250 m tiefen Bohrlöchern ca. 1 km unter Tage gelagert. Der mittlere Abstand solcher Bohrlöcher beträgt ca. 50 m auf einer Breite von 250 m und einer Länge von 1 - 2 km. Mit einer Wärmeproduktionsrate von etwas weniger als 1 kW für ein 1 m hohen Behälter kommt man etwa auf die obige mittlere Wärmeproduktionrate im gesamten Ablagerungsbereich.

Um das Problem mit Hilfe von Julia lösen zu können, müssen wir erst die nötigen Module (```ExtendableSparse, Plots```) und Submodule (```GeoModBox.HeatEquation.TwoD```) definieren: 

In [ ]:
using GeoModBox.HeatEquation.TwoD, ExtendableSparse, Plots
start = time()

Nun definieren wir die Model ($L$,$H$) und physikalischen Parameter ($k$,$Q$):

In [ ]:
# Physikalischer Parameter ---------------------------------------------- #
P       = ( 
    L       =   4.0e3,      #   [m]
    H       =   2.0e3,      #   [m]
    k       =   5.6,        #   Waermeleitfaehigkeit, W/m/K
    # Definiere die Region der Waermequelle
    Wcave   =   200.0,      # Breite der Region [ m ]
    Hcave   =   200.0,      # Mächtigkeit [ m ]
    Dcave   =   1.0e3,      # Tiefe des Zentrums [ m ]
    Xcave   =   2.0e3,      # x-Position des Zentrum [ m ]
    Q       =   0.3         # Volumetrische Waermeproduktionsrate [ W/m³ ]; Q = rho*H
)
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anzahl der Gitterpunkte und die Gitterabstände:

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   640,        # Gitterpunkte in x-Richtung
    y       =   320         # Gitterpunkte in y-Richtung    
    
)
Δ       = (
    x       =   P.L/NC.x,      #   Gitterabstand in x-Richtung
    y       =   P.H/NC.y       #   Gitterabstand in y-Richtung
)
# ----------------------------------------------------------------------- #

Mit deren Hilfe lässt sich das numerische Gitter bestimmen, so wie die Anfangsbedingungen unseres Problems: 

In [ ]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(0.0 + Δ.x[1]/2.0, P.L - Δ.x[1]/2.0, NC.x),
)
y       = (
    c       =   LinRange(-P.H + Δ.y[1]/2.0, 0.0 - Δ.y[1]/2.0, NC.y),
)
# ----------------------------------------------------------------------- #
# Erstellung der Anfangsbedingung --------------------------------------- #
D       = ( 
    Q       =   zeros(NC...),           # (row,col) 
    T       =   zeros(NC...),
)
# Definiere die Region der Anomalie ---
for i = 1:NC.x, j = 1:NC.y
    if x.c[i] >= (P.Xcave-P.Wcave/2.0) && x.c[i] <=(P.Xcave+P.Wcave/2.0) && 
        y.c[j] >= -P.Dcave-P.Hcave/2.0 && y.c[j] <= -P.Dcave+P.Hcave/2.0 
        D.Q[i,j]    = P.Q    
    end
end
# ----------------------------------------------------------------------- #

## Die Lösung 

### Diskretisierung

Um das Problem numerisch lösen zu können, müssen wir unsere Modeldomaine in ein numerisches Gitter unterteilen. Dabei nehmen wir an, das die Temperatur auf sogenannten *zentralen* Gitterpunkten (Centroids) definiert ist (siehe Abb. 2). Zur Lösung unseres Problems benutzen wir auch sogenannte *Ghost Nodes* in unserem Gitter, welche eine korrekte Einbindung der Randbedingungen ermöglichen. 

#### Gitter und Indizierung

<img src="../Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Abb. 2.** Versetztes Gitternetz. Die Temperatur ist auf den zenralen Gitterpunkten, den Centroids (rote Kreise) definiert. Die grauen Kreise außerhalb der Modelldomaine sind die Ghost Nodes. 

Das gegebene versetzte Gitter ermöglicht eine sogenannte *konservative* finite Differenzen Approximation, wobei angenommen wird, dass der Wärmefluss $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ auf dem Mittelpunkt der Gitterlinien und die Temperatur im Zentrum einer Gitterzelle definiert ist (streng genommen ist die Wärmeleitfähigkeit dann auch auf den Gitterlinien definiert; da diese allerdings konstant ist, müssen wir das hier nicht berücksichtigen; bei variablen thermischen Parameter muss die Gleichung etwas anders diskreditiert werden). Außerdem können wir durch die zentralen Temperaturgitterpunkte, in Verbindung mit den *Ghost Nodes* relativ einfach Randbedingungen einbinden, die die gleiche Fehlerordnung besitzen, wie die zentralen Differenzenquotienten im Inneren unseres Models. 

Bei der Indizierung unserer Gitterpunkte unterscheiden wir zwischen *lokalen* und *globalen* Indizes. Der lokale Index beschreibt die Position auf dem $i$,$j$-Gitter. Der globale Index, ist ein durchlaufender Index von 1 bis $nc_x \cdot nc_y$ und entspricht der Anzahl der Gleichungen, d.h. die Gesamtanzahl der inneren Gitterpunkte. Der globale Index wird auch beim Aufstellen der Koeffizientenmatrix für unser lineares Gleichungssystem verwendet. 

Für jeden Gitterpunkt, also für jede Gleichung, gibt ein sogenannter numerischer Stempel (*stencil*) die Position der Gitterpunkte an, welche für die jeweilige Gleichung gültig sind. Die Koeffizienten für diese Gitterpunkte sind dann jeweils ungleich null und alle anderen gleich null. Die Nomenklatur für den Stempel richtet sich häufig nach der eines Kompases, d.h. wir besitzen Punkte auf: Süden, Westen, Zentral, Osten, und Norden. Für jede Gleichung, ist der globale Index ($I$) jedes Punktes des Stempels gegeben durch die relative Position zum Zentral Punkt ($I^\textrm{C}$) des Stempels, d.h.: 

$\begin{equation}\begin{split}
I^\textrm{S} & = I^\textrm{C} - nc_x,\\
I^\textrm{W} & = I^\textrm{C} - 1,\\ 
I^\textrm{C} & = I, \\
I^\textrm{E} & = I^\textrm{C} + 1,\\
I^\textrm{N} & = I^\textrm{C} + nc_x,
\end{split}\end{equation}$

wobei $nc_x$ die Anzahl der horizontalen Centroids ist, $I^\textrm{C}$ ist der zentral Referenzpunkt, und $I^\textrm{S},I^\textrm{W},I^\textrm{E},I^\textrm{N}$ die Punkte im Süden, Westen, Osten, und Norden dazu sind

#### Finite Differenzen Approximation

Nun können wir die partielle Differentialgleichung durch unsere finiten Differenzen approximieren durch: 

$\begin{equation}
0 = \left( \frac{T_{I^\textrm{W}} - 2T_{I^\textrm{C}} + T_{I^\textrm{E}}}{\Delta{x}^2} + \frac{T_{I^\textrm{S}} - 2T_{I^\textrm{C}} + T_{I^\textrm{N}}}{\Delta{y}^2} \right) + \frac{Q}{k},
\end{equation}$

wobei $\Delta{x}, \Delta{y}$ die Gitterabstände in x- und y- Richtung sind. Durch Umformung erhalten wir ein lineares Gleichungsystem mit 5 Koeffizienten in der Form: 

$\begin{equation}
b T_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + 2b) T_{I^\textrm{C}} + a T_{I^\textrm{E}} + b T_{I^\textrm{N}} = - \frac{Q_{I^\textrm{C}}}{k},
\end{equation}$

wobei $ a = 1 / \Delta{x}^2$ und $b = 1 / \Delta{y}^2$ ist. 

### Randbedingungen

Die Temperatur auf den *Ghost Nodes* ist für *Dirichlet* und *Neumann* Randbedingungen genau so definiert, wie im 1-D Fall der [expliziten](./02_1D_Heat_explicit.ipynb) oder [impliziten](./03_1D_Heat_implicit.ipynb) Lösung der Wärmediffusionsgleichung (diesmal für vier Ränder statt für zwei). Da wir wieder ein lineares Gleichungssystem haben, müssen wir die Koeffizienten und die rechte Seite für die Gleichungen der **inneren Gitterpunkte** in der Nähe der Ränder, in Abhängigkeit der Randbedingungen, mit Hilfe der Temperatur auf den *Ghost Nodes* wie folgt modifizieren (Herleitung siehe Vorlesung): 

**Dirichlet** <br>
*West*
$\begin{equation}
bT_{I^\textrm{S}} - (3a + 2b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2aT_{BC}^W
\end{equation}$
*East*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (3a + 2b)T_{I^\textrm{C}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2aT_{BC}^E
\end{equation}$
*South*
$\begin{equation}
aT_{I^\textrm{W}} - (2a + 3b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2bT_{BC}^S
\end{equation}$
*North*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + 3b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} = -\frac{Q_{I^\textrm{C}}}{k} - 2bT_{BC}^N
\end{equation}$

**Neumann**<br>
*West*
$\begin{equation}
bT_{I^\textrm{S}} - (a + 2b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} + a c^W \Delta{x}
\end{equation}$
*East*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (a + 2b)T_{I^\textrm{C}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - a c^E \Delta{x}
\end{equation}$
*South*
$\begin{equation}
aT_{I^\textrm{W}} - (2a + b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} + b c^S \Delta{y}
\end{equation}$
*North*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} = -\frac{Q_{I^\textrm{C}}}{k} -b c^N \Delta{y}
\end{equation}$

Bei der Initialisierung der Randbedingungen bediehnung wir uns des Tuples `BC`, das die Art (`type`) und den Wert (`val`) der Randbedingung definiert. Die Temperatur auf den Ghost Noded wird dann später im Skript mit diesen berechnet.

In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (
    type    = (W=:Dirichlet, E=:Dirichlet, N=:Dirichlet, S=:Dirichlet),
    val     = (W=:0.0,E=:0.0,N=:0.0,S=:0.0)
)
# ----------------------------------------------------------------------- #

### Lösen der Problems

Gleichung $(5)$ ist ein Lineares Gleichungsystem der Form 

$\begin{equation}
\bm{K} T = T_I,
\end{equation}$

mit einer Koeffizientenmatrix $\bm{K}$ mit fünf nicht-null Diagonalen, der Anfangstemperaturbedingung $T_I$, und der stationären Lösung $T$. Daher definieren wir als nächstes die Parameter für unser lineares Gleichungssystem: 

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(Num.T)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# ----------------------------------------------------------------------- #

Das Aufstellen der Koeffizientenmatrix, die Modifikation der rechten Seite wie oben beschrieben, und die Lösung des Gleichungssystems werden in der Funktion ```Poisson!()``` durchgeführt. Durch die Einbindung des Submoduls ```GeoModBox.HeatEquation.TwoD``` kann die Funktion hier direkt aufgerufen werden (Achtet dabei auf die der Funktion mitzugebenen Parameter!). 

In [ ]:
# Solve equation -------------------------------------------------------- #
Poisson2Dc!(D,NC,P,BC,Δ,K,rhs,Num)
# ----------------------------------------------------------------------- #

Alternativ, könnte man auch die Rechenschritte aus der Funktion direkt hier programmieren, wobei man dann nicht mehr das Submodul laden muss. 

## Visualisierung

Zum Schluss plotten wir das Ergebniss noch: 

In [ ]:
# Plot solution --------------------------------------------------------- #
p = heatmap(x.c ./ 1e3, y.c ./ 1e3, D.T', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Stationary temperature field", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 900))

contour!(p, x.c ./ 1e3, y.c ./ 1e3, D.T', 
            levels=100:100:1500, linecolor=:black)

display(p)

savefig("./Results/04_Steady_State_Solution.png")
# ----------------------------------------------------------------------- #
stop = time()
println(stop-start)